In [ ]:
import xarray as xr
import intake
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cftime
import xesmf as xe
import pickle

# util.py is in the local directory
# it contains code that is common across project notebooks
# or routines that are too extensive and might otherwise clutter
# the notebook design
import util 
output_path='/home/jovyan/cmip6-outreach-tool/output/'

######### Define Settings for Data Dictionary
this_experiment_id = ['historical','ssp126', 'ssp370','ssp245','ssp585']
this_variable_id = 'tas'
this_table_id = 'Amon'
this_grid_label='gn'


######### Create Data Dictionary
import CreateDataDict
[dataset_info, dset_dict, modelnames]=CreateDataDict.createDataDict(this_experiment_id, this_variable_id, this_table_id, this_grid_label)

import xarray as xr
import intake
import pandas as pd
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cftime
import xesmf as xe
import pickle

# util.py is in the local directory
# it contains code that is common across project notebooks
# or routines that are too extensive and might otherwise clutter
# the notebook design
import util 
output_path='/home/jovyan/cmip6-outreach-tool/output/'

### Create ds_out reference file
def CreateReferenceGrid(modelname, experiment_id,activity_id):
    dataset_info_subset = dataset_info[dataset_info['source_id']==modelname]
    institution_id = list(set(dataset_info_subset['institution_id']))[0]
    nametag=activity_id+'.'+institution_id+'.'+modelname+'.'+experiment_id+'.'+this_table_id+'.'+this_grid_label
    thisdata=dset_dict[nametag]
    ds_out = xr.Dataset({'lat': thisdata['lat'],
                         'lon': thisdata['lon']})
    return ds_out

def RegridModel(thisdata,latvariable='lat',lonvariable='lon'):
    ds_in = xr.Dataset({'lat': thisdata[latvariable],
                    'lon': thisdata[lonvariable],
                    'time': thisdata['time'],
                    this_variable_id: thisdata[this_variable_id]})
    regridder = xe.Regridder(ds_in, ds_out, 'nearest_s2d')
    thisdata_regridded = regridder(ds_in)
    thisdata_regridded.attrs.update(thisdata.attrs)
    return thisdata_regridded

--> The keys in the returned dictionary of datasets are constructed as follows:
	'activity_id.institution_id.source_id.experiment_id.table_id.grid_label'

--> There will be 60 group(s)


In [ ]:
dict_timeSeries = dict()
experiment_id='historical'
activity_id='CMIP'
modelname='CAMS-CSM1-0'
ds_out = CreateReferenceGrid(modelname='CAMS-CSM1-0', activity_id = 'CMIP', experiment_id='historical')

In [5]:
for source_id in modelnames:
    dataset_info_subset = dataset_info[dataset_info['source_id']==source_id]
    institution_id = list(set(dataset_info_subset['institution_id']))[0]
    nametag = activity_id+'.'+institution_id+'.'+source_id+'.'+experiment_id+'.'+this_table_id+'.'+this_grid_label
    if nametag in dset_dict:
        print(modelname)
        thisdata=dset_dict[nametag]
        print(thisdata['time'])

CAMS-CSM1-0
<xarray.DataArray 'time' (time: 1980)>
array(['1850-01-16T12:00:00.000000000', '1850-02-15T00:00:00.000000000',
       '1850-03-16T12:00:00.000000000', ..., '2014-10-16T12:00:00.000000000',
       '2014-11-16T00:00:00.000000000', '2014-12-16T12:00:00.000000000'],
      dtype='datetime64[ns]')
Coordinates:
  * time     (time) datetime64[ns] 1850-01-16T12:00:00 ... 2014-12-16T12:00:00
Attributes:
    axis:           T
    bounds:         time_bnds
    long_name:      time
    standard_name:  time
CAMS-CSM1-0
<xarray.DataArray 'time' (time: 1980)>
array([cftime.DatetimeNoLeap(1850, 1, 16, 12, 0, 0, 0, 3, 16),
       cftime.DatetimeNoLeap(1850, 2, 15, 0, 0, 0, 0, 5, 46),
       cftime.DatetimeNoLeap(1850, 3, 16, 12, 0, 0, 0, 6, 75), ...,
       cftime.DatetimeNoLeap(2014, 10, 16, 12, 0, 0, 0, 6, 289),
       cftime.DatetimeNoLeap(2014, 11, 16, 0, 0, 0, 0, 2, 320),
       cftime.DatetimeNoLeap(2014, 12, 16, 12, 0, 0, 0, 4, 350)], dtype=object)
Coordinates:
  * time     (time) ob

In [13]:
dataset_info_subset = dataset_info[dataset_info['source_id']==modelname]
institution_id = list(set(dataset_info_subset['institution_id']))[0]
nametag=activity_id+'.'+institution_id+'.'+modelname+'.'+experiment_id+'.'+this_table_id+'.'+this_grid_label
thisdata=dset_dict[nametag]
thisdata = thisdata.mean(dim=['member_id'])
###### Reformat dates to be Proleptic Gregorian date type
#newtimes = reindex_time(startingtimes = thisdata['time'])
#thistime = xr.DataArray(newtimes, coords=[newtimes], dims=['time'])
#thisdata['time'] = thistime
###### Regrid 
thisdata=RegridModel(thisdata)
print(thisdata['time'])
#########################################
thisval=thisdata[this_variable_id] #.mean(dim=['lat','lon'])
ds = xr.Dataset({modelname: thisval},\
                coords={'time': thisdata['time']})
                        #'modelnames': modelnameInd, \
                        #'lat': thislat, \
                        #'lon': thislon, \
                

Overwrite existing file: nearest_s2d_160x320_160x320.nc 
 You can set reuse_weights=True to save computing time.
using dimensions ('lat', 'lon') from data variable tas as the horizontal dimensions for this dataset.
<xarray.DataArray 'time' (time: 1980)>
array([1.55000e+01, 4.50000e+01, 7.45000e+01, ..., 6.01485e+04, 6.01790e+04,
       6.02095e+04])
Coordinates:
  * time     (time) float64 15.5 45.0 74.5 ... 6.015e+04 6.018e+04 6.021e+04
Attributes:
    axis:           T
    bounds:         time_bnds
    calendar:       365_day
    long_name:      time
    standard_name:  time
    units:          days since 1850-01-01


In [ ]:
ds= initializeDataSet(activity_id,experiment_id,modelname='CAMS-CSM1-0')
[ds,modelnames_toplot] = fillDataSet()

In [ ]:
# Convert from K to C
for modelname in modelnames_toplot:
    ds[modelname]=ds[modelname]-273.15

#Add dataset to dictionary
dict_timeSeries[experiment_id] = ds
dict_timeSeries[experiment_id+'_modelnameToPlot'] = modelnames_toplot